In [ ]:
# API_TOKEN=<your_token>

In [ ]:
import os
import json
import requests
API_URL = "https://api-inference.huggingface.co/models/bigscience/bloom"
headers = {"Authorization": f"Bearer {API_TOKEN}"}
def query(payload):
    data = json.dumps(payload)
    response = requests.request("POST", API_URL, headers=headers, data=data)
    return json.loads(response.content.decode("utf-8"))

data = query(
    {"inputs":"Directional distance functions provide very flexible tools for investigating the performance of Decision Making Units (DMUs).",
    "parameters":{"seed":42,"early_stopping":False,"length_penalty":0,"max_new_tokens":128,"do_sample":True,"top_p":0.9}}
)

In [ ]:
data

In [ ]:
import numpy as np

def construct_passage(first_sentence, max_num_sentences=8):
    """
    Construct a passage from the first sentence.
    """
    passage_all = first_sentence
    range_var = np.random.randint(4, max_num_sentences)
    for _ in range(range_var):
        data = None
        try:
            # get the last 512 characters of the passages
            passage_that_fits = passage_all[-128:]
            data = query(
                {"inputs": passage_that_fits,
                "parameters":{"seed":45,"early_stopping":True,"length_penalty":0, "return_full_text": False, "max_new_tokens":64, "do_sample":True,"top_p":0.9}}
            )
            passage_all += data[0]["generated_text"]
        except Exception as e:
            print(e)
            if data:
                print(data)
    return passage_all.replace(first_sentence, '')

print(construct_passage("Directional distance functions provide very flexible tools for investigating the performance of Decision Making Units (DMUs)."))

In [ ]:
import pandas as pd
df = pd.read_csv(('./real_passages_100k.csv'))
samples = df.sample(n=1000, random_state=3).fillna("")

In [ ]:
!touch bloom_passages.csv

In [ ]:
from tqdm.auto import tqdm
from csv import writer
RESUME = 437
passages = []
j = 0
for i, sample in tqdm(samples.iterrows(), total=len(samples)):
    j += 1
    if j < RESUME:
        continue
    new_passage = construct_passage(sample['passages'])
    passages.append(
        new_passage
    )
    with open('bloom_passages.csv', 'a', newline='') as f_object:  
        writer_object = writer(f_object)
        writer_object.writerow([j, new_passage])  
        f_object.close()
    


In [ ]:
import pandas as pd
passages = pd.read_csv('bloom_passages.csv', header=0, names=['passage_number', 'passage']).fillna('')

In [ ]:
samples.iloc[0]

In [ ]:

from difflib import SequenceMatcher

SIMILARITY_THRESHOLD = 0.8

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
passages.iloc[2]['passage'].split('\n')

In [ ]:
clean = []
for i, passage in passages.iterrows():
    try:
        original = samples.iloc[passage['passage_number']]['passages']
        para = passage['passage']
        if para.strip() == '':
            continue
        if similar(para, original) >= SIMILARITY_THRESHOLD:
            continue
        
        sentences = para.split('\n')
        reconstructed_sentence = []
        for i, sentence in enumerate(sentences):
            if  i == len(sentences) - 1:
                reconstructed_sentence.append(sentence)
            elif similar(sentence, sentences[i+1]) >= 0.1:
                print(
                    sentence, sentences[i+1]
                )
            else:
                reconstructed_sentence.append(sentence)
        clean.append('\n'.join(reconstructed_sentence))
    except Exception as e:
        print(e)
        continue

In [ ]:
len(clean)

In [ ]:
clean

In [ ]:
df = pd.DataFrame({
    "passages": clean
})
df.to_csv('./bloom_generate.csv', index=False)

In [ ]:
clean[0]

In [ ]:
clean[2]